![](https://github.com/destination-earth/DestinE-DataLake-Lab/blob/main/img/DestinE-banner.jpg?raw=true)

# DEDL - Hook Tutorial

This notebook demonstrates how to use the Hook service.

The detailed API and definition of each endpoint and parameters is available in the OnDemand Processing API  Swagger UI at:
https://odp.data.destination-earth.eu/odata/docs

Further documentation is available at:
    https://destine-data-lake-docs.data.destination-earth.eu/en/latest/dedl-big-data-processing-services/Hook-service/Hook-service.html

### Install package and import environment variables 

In [17]:
#install DEDL authentification package to simplify your code
!pip install destine-auth

In [18]:
import json
#from io import BytesIO
#from urllib.parse import urlencode
#import pycurl
import requests
from getpass import getpass
import destine_auth as destine_auth

### Authentification - Get token

In [19]:
#Request DESP credentials
username = input("Please input your DESP username: ")
password = getpass("Please input your DESP password: ")

token = destine_auth.AuthHandler(DESP_USERNAME, DESP_PASSWORD)          
access_token = token.get_token()
 
# Check the status of the request
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
    #Save API headers
    api_headers = {'Authorization': 'Bearer ' + access_token}
else:
    print("Failed to Obtain DEDL/DESP Access Token") 

Please input your DESP username:  dana.puechmaille@gmail.com
Please input your DESP password:  ········


Response code: 200
DEDL/DESP Access Token Obtained Successfully


### Setup static variables

In [20]:
# Hook service url (ending with odata/v1/ - e.g. https://odp.data.destination-earth.eu/odata/v1/)
hook_service_root_url = "https://odp.data.destination-earth.eu/odata/v1/"


# List available workflows
Next we can check what possible workflows are available to us by using method   
```https://odp.data.destination-earth.eu/odata/v1/Workflows```

In [21]:
#Send request and return json object listing all provided workfows
result = requests.get(hook_service_root_url+"Workflows", headers=api_headers).json()

#Print provided workflows name
print("List of available provided Hook")
for i in range(len(result['value'])):            
    print(result['value'][i]['DisplayName']) # print JSON string 

List of available provided Hook
Sentinel-2: SNAP-Biophysical
Sentinel-1: Terrain-corrected backscatter (Private)
Sentinel-1 Coherence/Interferometry
Copernicus DEM Mosaic
Sentinel-2: MAJA Atmospheric Correction
Sentinel-2: C2RCC
Data harvest
ODP Test
DEDL Hello World
Sentinel-2: Sen2Cor


In [22]:
#Print JSON object: provided workflow list
#print(result, indent=3)

# Select a workflow and see parameters

If we want to see the details of a specific workflow, for example aditional parameters that can be changed we can do that too   
```https://odp.data.destination-earth.eu/odata/v1/Workflows?$expand=WorkflowOptions&$filter=(Name eq data-harvest)```   
**\\$expand=WorkflowOptions** shows all parameters accepted by workflow   
**\\$filter=(Name eq lai)** narrows the result to workflow called "lai" (processor computing Leaf Area Index)

In [58]:
#Select workflow 
workflow = "'data-harvest'" #example with data-haverst workflow

#Set Parameter filter
parameters = "Workflows?$expand=WorkflowOptions&$filter=(Name eq "

#Send request
result = requests.get(hook_service_root_url
                                  + parameters+workflow+")",
                                  headers=api_headers).json()
workflow_details = json.dumps(result, indent=2)
print(workflow_details) # print JSON string 



{'@odata.context': '$metadata#Workflows/$entity',
 'value': [{'Id': '11',
   'Uuid': None,
   'Name': 'data-harvest',
   'DisplayName': 'Data harvest',
   'Documentation': None,
   'Description': None,
   'InputProductType': None,
   'InputProductTypes': [],
   'OutputProductType': None,
   'OutputProductTypes': [],
   'WorkflowVersion': '0.0.1',
   'WorkflowOptions': [{'Name': 'input_catalogue_url',
     'Description': '',
     'Type': 'str',
     'Default': None,
     'Value': None,
     'Required': False},
    {'Name': 'input_catalogue_collection',
     'Description': '',
     'Type': 'str',
     'Default': None,
     'Value': None,
     'Required': False},
    {'Name': 'input_catalogue_type',
     'Description': "Catalogue type: one from ['STAC', 'ODATA']",
     'Type': 'CatalogueType',
     'Default': None,
     'Value': ['STAC', 'ODATA'],
     'Required': False},
    {'Name': 'source_server_url',
     'Description': '',
     'Type': 'str',
     'Default': None,
     'Value': None

## Order provided workflow: Data-

#Select workflow
workflow = "data-harvest"Make an order to harvest data using Harmonised Data Access API

In [24]:
#Select workflow
workflow = "data-harvest"

### Name your order

In [26]:
#ID of the run
order_name=input("Name your order: ")

Name your order:  TEST-DPu


### Define output storage

In workflow parameters, among others values, storage to retreive the result has to be provided.  
**Two possibilites:**
1. Use your user storage 
2. Use a temporary storage 

#### 1. - Your user storage (provided by DEDL ISLET service)

Example using a S3 bucket created with ISLET Storage service  - result will be available in this bucket
> *workflow parameter: {"Name": "output_storage", "Value": "PRIVATE"}*

In [27]:
# Output storage - Islet service
# URL of the S3 endpoint in the Central Site 
output_storage_url = "https://s3.central.data.destination-earth.eu"
# name of the object storage bucket where the results will be stored
output_bucket = "your-bucket-name"
# Islet object storage credentials (openstack ec2 credentials)
output_storage_access_key = "your-access-key"
output_storage_secret_key = "your-secret-key"
output_prefix = "dedl_" + order_name

#### 2 - Use temporary storage

The result of processing will be stored in shared storage and download link provided in the output product details
> *workflow parameter: {"Name": "output_storage", "Value": "TEMPORARY"}*

### Define parameters and send order

TODO: Update text
Now we can order a couple of products (we're ordering multiple products one by one to show the priority later, usually ordering multiple products can be done in a single call by using endpoint   
```https://odp.data.destination-earth.eu/odata/v1/BatchOrder/OData.CSC.Order```   

In [28]:
#Data have been previously discovered and search
STAC_HDA_API_URL = "https://hda.data.destination-earth.eu/stac"
COLLECTION_ID = "EO.ESA.DAT.SENTINEL-2.MSI.L2A"
print(STAC_HDA_API_URL+"/collections/"+COLLECTION_ID)

https://hda.data.destination-earth.eu/stac/collections/EO.ESA.DAT.SENTINEL-2.MSI.L2A


In [60]:
#data to retreive
data_id = "S2A_MSIL2A_20180124T092251_N0213_R093_T35TLG_20210214T010009.SAFE"

#Build your order body 
# order_body_custom_bucket = {
#         "Name": "DEDL - Hook tutorial - custom bucket - " + workflow + " - " + order_name,
#         "WorkflowName": workflow,
#         "IdentifierList": [data_id],
#         "WorkflowOptions":[
#             {"Name":"s3_bucket", "Value": output_bucket},
#             {"Name":"s3_access_key", "Value": output_storage_access_key},
#             {"Name":"s3_secret_key", "Value": output_storage_secret_key},
#             {"Name":"s3_prefix", "Value": output_prefix},
#             {"Name":"s3_endpoint_url", "Value": output_storage_url},
#             {"Name": "input_catalogue_type", "Value": "STAC"},
#             {"Name": "input_catalogue_url", "Value": "https://hda.data.destination-earth.eu/stac"},
#             {"Name": "input_catalogue_collection", "Value": "EO.ESA.DAT.SENTINEL-2.MSI.L2A"},
#             {"Name": "source_client_id", "Value": "hda-public"},
#             {"Name": "source_client_secret", "Value": ""},
#             {"Name": "source_username", "Value": username},
#             {"Name": "source_password", "Value": password},
#             {"Name": "source_realm", "Value": "dedl"},
#             {"Name": "source_server_url", "Value": "https://identity.data.destination-earth.eu/auth"}    
#         ]
#     }

#Order body example with temporary storage

order_body_custom_bucket = {
       "Name": "DEDL - Hook tutorial - temporary storage - " + workflow + " - " + order_name,
       "WorkflowName": workflow,
       "IdentifierList": [data_id],
       "WorkflowOptions":[
           {"Name": "output_storage", "Value": "TEMPORARY"},
           {"Name": "input_catalogue_type", "Value": "STAC"},
           {"Name": "input_catalogue_url", "Value": "https://hda.data.destination-earth.eu/stac"},
           {"Name": "input_catalogue_collection", "Value": "EO.ESA.DAT.SENTINEL-2.MSI.L2A"},
           {"Name": "source_client_id", "Value": "hda-public"},
           {"Name": "source_client_secret", "Value": ""},
           {"Name": "source_username", "Value": username},
           {"Name": "source_password", "Value": password},
           {"Name": "source_realm", "Value": "dedl"},
           {"Name": "source_server_url", "Value": "https://identity.data.destination-earth.eu/auth"}    
       ]
   }

order_request = requests.post(hook_service_root_url+"BatchOrder/OData.CSC.Order",
                            json.dumps(order_body_custom_bucket),headers=api_headers).json()

#If code = 201, the order has been successfully sent
print(order_request)

<module 'requests' from '/opt/conda/lib/python3.11/site-packages/requests/__init__.py'>

### Check The status of the order

Possible status
- queued (i.e. queued for treatment but not started)
- in_progress (i.e. order being treated)
- completed (i.e. order is complete and data ready)

In [64]:
requests_status = requests.get(hook_service_root_url + "ProductionOrders?$filter=(endswith(Name,'" + order_name + "'))", headers=api_headers).json()
requests_status #see requests status

{'@odata.context': '$metadata#ProductionOrder/$entity',
 'value': [{'Id': '11932',
   'Status': 'in_progress',
   'StatusMessage': 'request is under processing',
   'SubmissionDate': '2024-05-03T15:22:14.104Z',
   'Name': 'DEDL - Hook tutorial - temporary storage - data-harvest - TEST-DPu',
   'EstimatedDate': '2024-05-03T15:54:05.819Z',
   'InputProductReference': {'Reference': 'S2A_MSIL2A_20180124T092251_N0213_R093_T35TLG_20210214T010009.SAFE',
    'ContentDate': None},
   'WorkflowOptions': [{'Name': 'brand', 'Value': 'dedl'},
    {'Name': 'platform', 'Value': 'creodias'},
    {'Name': 'version', 'Value': '0.0.1'},
    {'Name': 'output_storage', 'Value': 'TEMPORARY'},
    {'Name': 'input_catalogue_type', 'Value': 'STAC'},
    {'Name': 'input_catalogue_url',
     'Value': 'https://hda.data.destination-earth.eu/stac'},
    {'Name': 'input_catalogue_collection',
     'Value': 'EO.ESA.DAT.SENTINEL-2.MSI.L2A'},
    {'Name': 'source_client_id', 'Value': 'hda-public'},
    {'Name': 'source

### Access workflow output

#### Private storage
Let us now check our private storage using this boto3 script.
You can also go and check this in the Islet service using the Horizon user interface

In [ ]:
import boto3

s3 = boto3.client('s3',aws_access_key_id=output_storage_access_key, aws_secret_access_key=output_storage_secret_key, endpoint_url=output_storage_url,)

paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=output_bucket, Prefix=output_prefix + '/')

for page in pages:
    try:
        for obj in page['Contents']:
            print(obj['Key'])
    except KeyError:
        print("No files exist")
        exit(1)

### Temporary storage

In [68]:
# replace XXXX with order id of a completed order from the current test run
# list order items within a production order

#requests.get('https://odp.data.destination-earth.eu/odata/v1/BatchOrder(11928)/Products', headers=api_headers).json()

{'@odata.context': '#metadata/OData.CSC.BatchorderItem',
 'value': [{'Id': 15321,
   'BatchOrderId': 11928,
   'InputProductReference': 'S2A_MSIL2A_20180124T092251_N0213_R093_T35TLG_20210214T010009.SAFE',
   'SubmissionDate': '2024-05-03T15:10:25.090Z',
   'Status': 'in_progress',
   'ProcessedName': None,
   'ProcessedSize': None,
   'OutputUUID': None,
   'DownloadLink': None,
   'NotificationStatus': None,
   'CompletedDate': None}]}

In [ ]:
# replace XXXX with order id of a completed order from the current test run and order item id YYYY from the item listing
# download output product
# result is stored in output.zip and number of transferred bytes is printed

#url = 'https://odp.data.destination-earth.eu/odata/v1/BatchOrder(XXXX)/Product(YYYY)/$value'
#r = requests.get(url, headers=api_headers, allow_redirects=True)

#open('output.zip', 'wb').write(r.content)